# Image classification with Convolutional Neural Networks

In [29]:
!python -V

Python 3.6.5 :: Anaconda, Inc.


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [5]:
torch.cuda.is_available()

True

In [6]:
torch.backends.cudnn.enabled

True

# Import data

In [31]:
import json
import urllib.request
import shutil
from collections import defaultdict

In [7]:
PATH = "data/imaterialist-challenge-furniture-2018"

In [8]:
# !kaggle competitions download -c 'imaterialist-challenge-furniture-2018' -p {PATH}

In [9]:
!ls {PATH}

sample_submission_randomlabel.csv  train       validation.json
test.json			   train.json


In [11]:
with open(f'{PATH}/train.json') as json_data:
    train_data = json.load(json_data)

In [13]:
sum(train_data)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [30]:
train_data.keys()

dict_keys(['images', 'annotations'])

In [17]:
train_img_path = f'{PATH}/train'
os.makedirs(train_img_path, exist_ok=True)

In [18]:
!ls {PATH}

sample_submission_randomlabel.csv  train       validation.json
test.json			   train.json


In [61]:
image_labels = {}
labels = set()
for img in train_data['annotations']:
    image_labels[img['image_id']] = img['label_id']
    labels.add(img['label_id'])

In [60]:
len(image_labels)

194828

In [63]:
len(labels)

128

In [66]:
for label in labels:
    os.makedirs(f'{train_img_path}/{label}', exist_ok=True)

In [69]:
!ls {train_img_path}

1    106  113  120  128  2   27  34  41  49  56  63  70  78  85  92
10   107  114  121  13	 20  28  35  42  5   57  64  71  79  86  93
100  108  115  122  14	 21  29  36  43  50  58  65  72  8   87  94
101  109  116  123  15	 22  3	 37  44  51  59  66  73  80  88  95
102  11   117  124  16	 23  30  38  45  52  6	 67  74  81  89  96
103  110  118  125  17	 24  31  39  46  53  60  68  75  82  9	 97
104  111  119  126  18	 25  32  4   47  54  61  69  76  83  90  98
105  112  12   127  19	 26  33  40  48  55  62  7   77  84  91  99


In [84]:
test_count = 10
count = 0
for image in train_data['images']:
    url = image['url'][0]
    id = image['image_id']
    file_extension = url.split('.')[-1]
    file_path = f'{train_img_path}/{image_labels[id]}/{id}.{file_extension}'
    with urllib.request.urlopen(url) as response, open(file_path, 'wb') as out_file:
        shutil.copyfileobj(response, out_file)

    count = count + 1
    if count > test_count:
        break